In [4]:
import sympy as s
import numpy as np

In [144]:
omega = s.Symbol("omega") #freq harmonic oscillator
eta = s.Symbol("eta") #efficiency measurement (appearing in the CPTP decomposiion X,Y)
N = s.Symbol("N") #mean number of excitations in system B (environment, thermal state)

In [145]:
Omega = s.Matrix(np.array([[0,1],[-1,0]]))

In [146]:
## interaction hamiltonian between the train of modes being measured and system

Cint = -Omega # -\ii \Sigma_y
s.Matrix(Omega)*(s.Matrix(Cint))
Hint = np.block([[s.zeros(2), Cint], [Cint.T, s.zeros(2)]])/2

xs, ps, xb, pb = s.Symbol("xs"), s.Symbol("ps"), s.Symbol("xb"), s.Symbol("pb")

print((s.Matrix([xs,ps,xb,pb]).T)*Hint*s.Matrix([xs,ps,xb,pb])) #BS splitter


aint = Omega*Cint*Omega*(Cint.T)/2

Hs = s.eye(2)*omega
asys = Omega*Hs

A_matrix = aint + asys ##drift matrix

Matrix([[-pb*xs + ps*xb]])


In [147]:
#noisy heterodyne measurement
S = s.eye(2) 
sigma_m = S*(S.T)
##lossy part
Xstar = s.eye(2)/s.sqrt(eta)
Ystar = s.eye(2)*(1-eta)/eta
sigma_noisy_m = Xstar*(sigma_m)*(Xstar.T) + Ystar

# state of B, subsystem being measured
sigma_b = (2*N +1)*s.eye(2)

D_matrix = Omega*Cint*sigma_b*(Omega*Cint).T  #difussion matrix
Gamma_matrix = Omega*Cint*sigma_b   #auxiliar matrix for the ricatti

C_matrix = (Cint*Omega*(sigma_b + sigma_noisy_m)**(-1/2)).T

 Ricatti equation for covariance

In [125]:
from scipy.linalg import solve_continuous_are
A_tilde_matrix = A_matrix - (Omega*Cint*sigma_b*((sigma_b + sigma_noisy_m)**(-1))*Omega*Cint.T   )
D_tilde_matrix = D_matrix + (Omega*Cint*sigma_b*((sigma_b + sigma_noisy_m)**(-1))*sigma_b*Cint.T*Omega)
B_matrix = Cint*Omega*(sigma_b + sigma_noisy_m)**(-1/2)

In [158]:
Ar, Br, Dr = [np.array(k.subs({omega:2*np.pi, N:2, eta:0.8})).astype(np.float32) for k in [A_tilde_matrix, B_matrix, D_tilde_matrix]]

In [164]:
cov_stat = solve_continuous_are(Ar, Br, Dr, np.eye(2))

In [166]:
varx, varp, covxp = [s.Symbol(k) for k in ["varx","varp","covxp"]]
cov = s.Matrix([[varx, covxp],[covxp, varp]])

In [181]:
(A_tilde_matrix*cov + cov*A_tilde_matrix.T + D_tilde_matrix - B_matrix*B_matrix.T).values()

[2*N + 2*covxp*omega + 2*varx*((2*N + 1)/(2*N + 1 + (1 - eta)/eta + 1/eta) - 1/2) - (2*N + 1)**2/(2*N + 1 + (1 - eta)/eta + 1/eta) - (2*N + 1 + (1 - eta)/eta + 1/eta)**(-1.0) + 1,
 2*covxp*((2*N + 1)/(2*N + 1 + (1 - eta)/eta + 1/eta) - 1/2) + omega*varp - omega*varx,
 2*covxp*((2*N + 1)/(2*N + 1 + (1 - eta)/eta + 1/eta) - 1/2) + omega*varp - omega*varx,
 2*N - 2*covxp*omega + 2*varp*((2*N + 1)/(2*N + 1 + (1 - eta)/eta + 1/eta) - 1/2) - (2*N + 1)**2/(2*N + 1 + (1 - eta)/eta + 1/eta) - (2*N + 1 + (1 - eta)/eta + 1/eta)**(-1.0) + 1]

In [ ]:
def Euler(ff, G, y0, tspan, dt,**kwargs):
    exp = kwargs.get("exp",False)
    N = len(tspan)
    y = np.zeros((N, len(y0)))
    y[0] = y0

    if exp is True:
        dint = 1.
    else:
        dint= dt

    for ind,t in enumerate(tqdm(tspan[:-1])):
        #dW = np.random.normal(0,np.sqrt(dt), (7))
        w0 = np.random.normal()*np.sqrt(dt)
        w1 = np.random.normal()*np.sqrt(dt)
        dW = np.array([w0, w1, w0, w1 , 0.,0.,0.])
        y[ind+1] = y[ind] + ff(y[ind], t, exp=exp, dt=dt)*dint + np.dot(G(y[ind], t), dW)
    return y